# Simulating quantum chemistry with `pennylane-qchem`

created by G. K. Sunnardianto [gagu001(at)brin.go.id]() and M. Y. Hanna [muha207(at)brin.go.id]()


---

## Before we start

## Simulating H2 molecule

This Python code uses the PennyLane library to perform a quantum computation. The goal is to find the ground state energy of a hydrogen molecule (H2) using the Variational Quantum Eigensolver (VQE) algorithm.

#### Construct the molecule

In [ ]:
# This alias allows you to refer to the PennyLane module using the shorter name qml
import pennylane as qml
# qchem submodule provides tools for quantum chemistry simulations
from pennylane import qchem
# numerical computing and plotting library
from pennylane import numpy as np
import matplotlib.pyplot as plt
plt.style.use('../imag/SciencePlots-APS.mplstyle')

# define atomic symbols
symbols = ["H", "H"]
# define coordinates
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

#### Compute required qubits

In [ ]:
# compute the molecular Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
#print("The Hamiltonian is ", H)

Defining the device by using a qubit simulator Here we used `default.qubit`; we can also use other devices such as `lightning.qubit`. Please visit [this link](https://pennylane.ai/plugins/) to see more about the pennylane device.

![pennylane-device](../imag/pennylane-device.png)

#### Generate vector via the Hartree-Fock state


We generate the vector representing the Hartree-Fock state as an ansatz using the `hf_state` function. An ansatz is an educated assumption about the value or form of an unknown function and plays a crucial role in deriving the real solution of an equation. 

In [ ]:
# Defining the device by using qubit simulator
dev = qml.device("default.qubit", wires=qubits)

# Number of electrons
electrons = 2

# We use the `hf_state` function to generate the Hartree-Fock state
hf = qml.qchem.hf_state(electrons, qubits)

# Print the Hartree-Fock state
print(hf)

The `hf_state` is used to initialize the qubit register. Then, we just act with the `DoubleExcitation` operation on the four qubits.

![jordan-wigner](../imag/jordan-wigner.png)

In [ ]:
def circuit(param, wires):
    # Initialize the circuit to the Hartree-Fock state
    qml.BasisState(hf, wires=wires)
    
    # Apply a double excitation operation to the first four qubits
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

Define the cost function to compute the expectation
value of the molecular Hamiltonian in the trial state prepared by the
circuit. The decorator syntax allows us to run the cost function as an
executable QNode with the gate parameter $\theta$

In [ ]:
# The decorator syntax, the decorator specifies the quantum device (dev) on which the quantum computation will be executed.
@qml.qnode(dev)
def cost_fn(param):  # Define the cost function
    # Apply the quantum circuit with the given parameters to the specified number of qubits
    circuit(param, wires=range(qubits))
    # Return the expectation value of the Hamiltonian (H)
    return qml.expval(H)

Now we proceed to minimize the cost function to find the ground state of
the $\mathrm{H}_2$ molecule. He we used classical optimizer: Gradient Descent Optimizer
We initialize the circuit parameter $\theta$ to zero, meaning that we
start from the Hartree-Fock state.



In [ ]:
# Instantiate the classical optimizer: Gradient Descent Optimizer with a step size of 0.4
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# Initialize the parameter theta for performing gradient-based optimization, set to 0.0 and marked as requiring gradient computations
theta = np.array(0.0, requires_grad=True)

We carry out the optimization over a maximum of 100 steps aiming to
reach a convergence tolerance of $10^{-6}$ for the value of the cost
function.

In [ ]:
# Instantiate the classical optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# Initialize the list to store the values of the cost function
energy = [cost_fn(theta)]

# Initialize the list to store the values of the circuit parameter
angle = [theta]

# Set the maximum number of optimization iterations
max_iterations = 100

# Set the convergence tolerance
conv_tol = 1e-06

# Start the optimization loop
for n in range(max_iterations):
    # Update the parameter theta and get the previous energy value
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    # Append the updated cost function value to the energy list
    energy.append(cost_fn(theta))

    # Append the updated parameter value to the angle list
    angle.append(theta)

    # Calculate the absolute difference between the current energy value and the previous energy value
    conv = np.abs(energy[-1] - prev_energy)

    # Print the optimization progress every two steps
    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    # If the convergence criterion is met, break the loop
    if conv <= conv_tol:
        break

# Print the final value of the ground-state energy
print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")

# Print the optimal value of the circuit parameter
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")

Let\'s plot the values of the ground state energy of the molecule

In [ ]:
# Full configuration interaction (FCI) energy computed classically
E_fci = -1.136189454088

fig = plt.figure(figsize=(5,5))

plt.plot(range(len(energy)), energy, 'o', ls='dashed', color='blue')
plt.xlabel('Optimization step')
plt.axhline(E_fci, color='red', ls='--')
plt.ylabel('Energy (Hartree)')
plt.show()

##### The resulting energy matches the exact energy of the ground electronic state of H2, which is -1.1357 Ha
---

## Simulating LiH

we first need to define the molecular parameters, including atomic symbols and coordinates

In [ ]:
# Define atomic symbols for Lithium (Li) and Hydrogen (H)
symbols = ["Li", "H"] 
# Define coordinates for the atoms in the molecule
geometry = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 2.969280527]) 

Compute the molecular Hamiltonian via `qchem.molecular_hamiltonian`  & obtain the electronic excitations via "`qchem.excitations`". `qchem.excitations` function  will computes the electronic excitations between different molecular orbitals.

In [ ]:
# Define the molecular Hamiltonian
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=2, # The number of active electrons 
    active_orbitals=5 # The number of active molecular orbitals
)

# Set the number of active electrons
active_electrons = 2

# Get the single and double excitations
singles, doubles = qchem.excitations(active_electrons, qubits) 

# Print the total number of excitations
print(f"Total number of excitations = {len(singles) + len(doubles)}")

We create **single** and **double excitation** operators and then combines them into a pool of operators

In [ ]:
# Create a list of single excitations with initial amplitude 0.0
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]

# Create a list of double excitations with initial amplitude 0.0
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]

# Combine the single and double excitations into an operator pool
operator_pool = doubles_excitations + singles_excitations

We define an initial circuit that prepares a Hartree-Fock state. Define a device and calculate the expectation value of the Hamiltonian. The `qchem.hf_state` function is used to generate the Hartree-Fock (HF) state for a quantum chemistry simulation

In [ ]:
# Generate the initial state (Hartree-Fock state)
hf_state = qchem.hf_state(active_electrons, qubits)

# Define a quantum device
dev = qml.device("default.qubit", wires=qubits)

# Define a quantum node (QNode)
@qml.qnode(dev)
def circuit():
    # Apply Pauli-X gates to the qubits for which the Hartree-Fock state is non-zero
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    # Return the expectation value of the Hamiltonian
    return qml.expval(H)

In [ ]:
# Initialize an AdaptiveOptimizer
opt = qml.optimize.AdaptiveOptimizer()

# Initialize lists to store energy and step values
energies = []
steps = []

# Iterate over the elements in the operator_pool
for i in range(len(operator_pool)):
    # Build the circuit adaptively and perform one optimization step
    # Also calculate the energy and gradient
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)

    # Store energy and step values
    energies.append(energy)
    steps.append(i)

    # Print the energy, largest gradient, and circuit every 3 steps
    if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        # Draw the current state of the quantum circuit
        print(qml.draw(circuit, decimals=None)())
        print()

    # If the largest gradient in the optimization is below a certain threshold (3e-3), 
    # terminate the optimization loop early
    if gradient < 3e-3:
        break

In [ ]:
# Full configuration interaction (FCI) energy computed classically
E_fci = -7.8825378193

fig = plt.figure(figsize=(5, 5))

plt.plot(steps, energies, 'o', ls='dashed', color='blue')

plt.xlabel('Optimization step')
plt.axhline(E_fci, color='red', ls='--')
plt.ylabel('Energy (Hartree)')
plt.show()

##### The resulting energy matches the exact energy of the ground electronic state of LiH, which is -7.8825378193 Ha